In [1]:
# analise_dados.py

import chart_studio.plotly as py
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
import plotly.io as pio
pio.renderers.default = "vscode"

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings

# Importa todas as funções do arquivo funcoes_analise.py
from funcoes_analise import *

# Suprime avisos para manter a saída limpa
warnings.filterwarnings('ignore')

In [2]:
# --- Seção 1: Carregamento e Preparação dos Dados Medidos ---
print(">>> Iniciando Análise de Dados Medidos <<<\n")

# Carrega o arquivo de dados. 
# Certifique-se de que o caminho 'received_power_doisandares.txt' está correto.
df_real = pd.read_csv('received_power_doisandares.txt')

# Carrega e processa os dados medidos no térreo (andar == 0)
# CORREÇÃO: Usado .loc[] com colchetes e .copy() para evitar avisos.
df_real_terreo = df_real.loc[df_real['andar'] == 0].copy()
df_real_terreo['pl'] = 20 - df_real_terreo['Cp']  # Potência de transmissão de 20 dBm

# Carrega e processa os dados medidos no 1º andar (andar == 1)
# CORREÇÃO: Usado .loc[] com colchetes e .copy() para evitar avisos.
df_real_1andar = df_real.loc[df_real['andar'] == 1].copy()

# A função calcdist precisa estar definida no seu código
# df_real_1andar['distancia'] = calcdist(df_real_1andar['distancia'] + 1) 
df_real_1andar['pl'] = 20 - df_real_1andar['Cp']

# Verificação dos resultados
print("--- Dados do Térreo Processados ---")
print(df_real_terreo.head())

print("\n--- Dados do 1º Andar Processados ---")
print(df_real_1andar.head())

>>> Iniciando Análise de Dados Medidos <<<

--- Dados do Térreo Processados ---
   distancia     Cp  andar     pl
0          1 -22.77      0  42.77
1          2 -29.06      0  49.06
2          3 -30.70      0  50.70
3          4 -34.75      0  54.75
4          5 -34.74      0  54.74

--- Dados do 1º Andar Processados ---
    distancia     Cp  andar     pl
39          1 -53.64      1  73.64
40          2 -50.12      1  70.12
41          3 -42.56      1  62.56
42          4 -46.67      1  66.67
43          5 -50.95      1  70.95


In [3]:
# --- Seção 2: Visualização dos Dados Medidos ---

# Gráfico 1: Perfil de Potência Recebida (Pr)
fig_pr = go.Figure()
fig_pr.add_trace(go.Scatter(x=df_real_terreo.distancia, y=df_real_terreo.Cp, name='Pr Térreo', mode='markers'))
fig_pr.add_trace(go.Scatter(x=df_real_1andar.distancia, y=df_real_1andar.Cp, name='Pr 1º Andar', mode='lines+markers'))
fig_pr.update_layout(title='Perfil de Potência Recebida (Medido)', xaxis_title='Distância (m)', yaxis_title='Potência Recebida (dBm)')
fig_pr.show()


In [4]:
# Gráfico 2: Perda de Percurso (Path Loss)
fig_pl = go.Figure()
fig_pl.add_trace(go.Scatter(x=df_real_terreo.distancia, y=df_real_terreo['pl'], name='Path Loss Térreo', mode='markers'))
fig_pl.add_trace(go.Scatter(x=df_real_1andar.distancia, y=df_real_1andar['pl'], name='Path Loss 1º Andar', mode='lines+markers'))
fig_pl.update_layout(title='Path Loss (Medido)', xaxis_title='Distância (m)', yaxis_title='Path Loss (dB)')
fig_pl.show()


In [5]:
# --- Seção 3: Análise com Modelos Empíricos ---
print("\n>>> Iniciando Análise com Modelos Empíricos <<<\n")

# Análise para o Térreo
print("--- Resultados para o Térreo ---")
fi_terreo, _, _ = float_intercept(df_real_terreo[['distancia', 'Cp', 'pl']])
ci_terreo, _ = close_in(df_real_terreo[['distancia', 'Cp', 'pl']])

fig_modelos_terreo = go.Figure()
fig_modelos_terreo.add_trace(go.Scatter(x=df_real_terreo.distancia, y=df_real_terreo['pl'], name='Dados Medidos', mode='markers'))
fig_modelos_terreo.add_trace(go.Scatter(x=df_real_terreo.distancia, y=fi_terreo, name='Modelo Float Intercept', mode='lines+markers'))
fig_modelos_terreo.add_trace(go.Scatter(x=df_real_terreo.distancia, y=ci_terreo, name='Modelo Close-In', mode='lines+markers'))
fig_modelos_terreo.update_layout(title='Modelos de Path Loss vs. Medido (Térreo)', xaxis_title='Distância (m)', yaxis_title='Path Loss (dB)')
fig_modelos_terreo.show()


>>> Iniciando Análise com Modelos Empíricos <<<

--- Resultados para o Térreo ---
--- Modelo Floating Intercept (FI) ---
RMSE: 2.6902
Alpha: 48.2405
Beta: 0.8832
Shadowing (Std Dev): 2.6902

--- Modelo Close-In (CI) ---
RMSE: 3.0684
Path Loss Exponent (PLE): 1.2600
Shadowing (Std Dev): 3.0363


In [6]:
# Análise para o 1º Andar
print("\n--- Resultados para o 1º Andar ---")
fi_1andar, _, _ = float_intercept(df_real_1andar[['distancia', 'Cp', 'pl']])
ci_1andar, _ = close_in(df_real_1andar[['distancia', 'Cp', 'pl']])

fig_modelos_1andar = go.Figure()
fig_modelos_1andar.add_trace(go.Scatter(x=df_real_1andar.distancia, y=df_real_1andar['pl'], name='Dados Medidos', mode='markers'))
fig_modelos_1andar.add_trace(go.Scatter(x=df_real_1andar.distancia, y=fi_1andar, name='Modelo Float Intercept', mode='lines+markers'))
fig_modelos_1andar.add_trace(go.Scatter(x=df_real_1andar.distancia, y=ci_1andar, name='Modelo Close-In', mode='lines+markers'))
fig_modelos_1andar.update_layout(title='Modelos de Path Loss vs. Medido (1º Andar)', xaxis_title='Distância (m)', yaxis_title='Path Loss (dB)')
fig_modelos_1andar.show()


--- Resultados para o 1º Andar ---
--- Modelo Floating Intercept (FI) ---
RMSE: 4.2109
Alpha: 63.8694
Beta: 1.2620
Shadowing (Std Dev): 4.2109

--- Modelo Close-In (CI) ---
RMSE: 7.4967
Path Loss Exponent (PLE): 2.8488
Shadowing (Std Dev): 7.2591


In [ ]:
# --- Seção 4: Comparação com Simulação Full3D ---
print("\n>>> Comparação com Dados Simulados (Full3D) <<<\n")

# Processa e carrega dados simulados do térreo
txt_to_csv('caminho/para/ProjetoLCT-TCC.power.t001_01.r003.p2m', 'sim_full3d_terreo.csv', is_full3d=True) # <-- SUBSTITUA O CAMINHO
df_sim_terreo = pd.read_csv('sim_full3d_terreo.csv')
df_sim_terreo['Distance (m)'] += 1
df_sim_terreo['pl'] = 20 - df_sim_terreo['Power (dBm)']

# Processa e carrega dados simulados do 1º andar
txt_to_csv('caminho/para/ProjetoLCT-TCC.power.t001_01.r004.p2m', 'sim_full3d_1andar.csv', is_full3d=True) # <-- SUBSTITUA O CAMINHO
df_sim_1andar = pd.read_csv('sim_full3d_1andar.csv')
df_sim_1andar['pl'] = 20 - df_sim_1andar['Power (dBm)']
# ATENÇÃO: A distância da simulação está sendo substituída pela distância medida para comparação direta.
df_sim_1andar['Distance (m)'] = df_real_1andar['distancia']

# Calcula o RMSE entre o medido e o simulado
rmse_terreo = rmse(df_real_terreo['pl'], df_sim_terreo['pl'])
rmse_1andar = rmse(df_real_1andar['pl'], df_sim_1andar['pl'])
print(f"RMSE (Medido vs. Simulado Full3D) - Térreo: {rmse_terreo:.4f}")
print(f"RMSE (Medido vs. Simulado Full3D) - 1º Andar: {rmse_1andar:.4f}\n")

In [ ]:
# Gráfico de comparação - Térreo
plt.figure(figsize=(10, 6))
plt.scatter(df_real_terreo['distancia'], df_real_terreo['pl'], marker='*', color='red', label='PL Medido')
plt.plot(df_sim_terreo['Distance (m)'], df_sim_terreo['pl'], marker='o', linestyle='-', label='PL Simulado (Full3D)')
plt.title(f'Path Loss Térreo: Medido vs. Simulado (RMSE = {rmse_terreo:.4f})')
plt.xlabel('Distância (m)')
plt.ylabel('Path Loss (dB)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Gráfico de comparação - 1º Andar
plt.figure(figsize=(10, 6))
plt.scatter(df_real_1andar['distancia'], df_real_1andar['pl'], marker='*', color='red', label='PL Medido')
plt.plot(df_sim_1andar['Distance (m)'], df_sim_1andar['pl'], marker='o', linestyle='-', label='PL Simulado (Full3D)')
plt.title(f'Path Loss 1º Andar: Medido vs. Simulado (RMSE = {rmse_1andar:.4f})')
plt.xlabel('Distância (m)')
plt.ylabel('Path Loss (dB)')
plt.legend()
plt.grid(True)
plt.show()